# A guide Portfolio Optimization Environment

This notebook aims to provide an example of using PortfolioOptimizationEnv (or POE) to train a reinforcement learning model that learns to solve the portfolio optimization problem.

In this document, we will reproduce a famous architecture called EIIE (ensemble of identical independent evaluators), introduced in the following paper:

- Zhengyao Jiang, Dixing Xu, & Jinjun Liang. (2017). A Deep Reinforcement Learning Framework for the Financial Portfolio Management Problem. https://doi.org/10.48550/arXiv.1706.10059.

It's advisable to read it to understand the algorithm implemented in this notebook.

### Note
If you're using this environment, consider citing the following paper (in adittion to FinRL references):

- Caio Costa, & Anna Costa (2023). POE: A General Portfolio Optimization Environment for FinRL. In *Anais do II Brazilian Workshop on Artificial Intelligence in Finance* (pp. 132–143). SBC. https://doi.org/10.5753/bwaif.2023.231144.

```
@inproceedings{bwaif,
 author = {Caio Costa and Anna Costa},
 title = {POE: A General Portfolio Optimization Environment for FinRL},
 booktitle = {Anais do II Brazilian Workshop on Artificial Intelligence in Finance},
 location = {João Pessoa/PB},
 year = {2023},
 keywords = {},
 issn = {0000-0000},
 pages = {132--143},
 publisher = {SBC},
 address = {Porto Alegre, RS, Brasil},
 doi = {10.5753/bwaif.2023.231144},
 url = {https://sol.sbc.org.br/index.php/bwaif/article/view/24959}
}

```

## Installation and imports

To run this notebook in google colab, uncomment the cells below.

In [ ]:
## install finrl library
# !pip install wrds
# !pip install quantstats
# !pip install torch_geometric
# !pip install swig
# !pip install -q condacolab
# !pip install shimmy
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/flpymonkey/FinRL_Online_Portfolio_Benchmarks.git

In [114]:
## Hide matplotlib warnings
# import warnings
# warnings.filterwarnings('ignore')

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

#### Import the necessary code libraries

In [115]:
import torch

import numpy as np

from sklearn.preprocessing import MaxAbsScaler

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Fetch data

In his paper, *Jiang et al* creates a portfolio composed by the top-11 cryptocurrencies based on 30-days volume. Since it's not specified when this classification was done, it's difficult to reproduce, so we will use a similar approach in the Brazillian stock market:

- We select top-10 stocks from Brazillian stock market;
- For simplicity, we disconsider stocks that have missing data for the days in period 2011-01-01 to 2019-12-31 (9 years);

In [ ]:
TEST_TICKER = [
   "MSFT",
    "V",
    "AAPL",
    "BA",
    "INTC",
    "WMT",
]


TRAIN_START_DATE = '2009-04-01'
TRAIN_END_DATE = '2021-12-31'
TEST_START_DATE = '2022-01-01'
TEST_END_DATE = '2024-10-01'

TRAIN_DATES = ['2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31', '2023-12-31']


In [117]:
print(len(TEST_TICKER))

portfolio_raw_df = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TEST_END_DATE,
                                ticker_list = TEST_TICKER).fetch_data()
portfolio_raw_df

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

6



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (22278, 8)


,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.274470,589372000,AAPL,2
1,2009-04-01,34.520000,35.599998,34.209999,26.850744,9288800,BA,2
2,2009-04-01,14.770000,15.320000,14.620000,9.507450,75052800,INTC,2
3,2009-04-01,18.230000,19.360001,18.180000,14.301779,96438900,MSFT,2
4,2009-04-01,13.687500,14.000000,13.407500,12.129699,44144400,V,2
...,...,...,...,...,...,...,...,...
22273,2023-12-29,260.670013,262.220001,259.559998,260.660004,3681900,BA,4
22274,2023-12-29,50.299999,50.570000,49.770000,49.585903,29266500,INTC,4
22275,2023-12-29,376.000000,377.160004,373.480011,373.248871,18723000,MSFT,4
22276,2023-12-29,260.570007,261.459991,259.670013,258.375092,4074600,V,4


In [118]:
portfolio_raw_df.groupby("tic").count()

,date,open,high,low,close,volume,day
tic,,,,,,,
AAPL,3713,3713,3713,3713,3713,3713,3713
BA,3713,3713,3713,3713,3713,3713,3713
INTC,3713,3713,3713,3713,3713,3713,3713
MSFT,3713,3713,3713,3713,3713,3713,3713
V,3713,3713,3713,3713,3713,3713,3713
WMT,3713,3713,3713,3713,3713,3713,3713


### Normalize Data

We normalize the data dividing the time series of each stock by its maximum value, so that the dataframe contains values between 0 and 1.

In [119]:
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(portfolio_raw_df)
portfolio_norm_df

c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.31332941 0.43188074 0.33824661 ... 0.02556499 0.01810204 0.02266286]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5  0.75 1.   ... 0.5  0.75 1.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


,date,open,high,low,close,volume,tic,day
0,2009-04-01,0.018773,0.019501,0.018834,0.016610,0.313329,AAPL,0.5
1,2009-04-01,0.077397,0.079819,0.077716,0.062400,0.089997,BA,0.5
2,2009-04-01,0.216569,0.221100,0.217204,0.153140,0.377145,INTC,0.5
3,2009-04-01,0.047504,0.050377,0.048075,0.037650,0.302015,MSFT,0.5
4,2009-04-01,0.052006,0.053181,0.051633,0.046583,0.130785,V,0.5
...,...,...,...,...,...,...,...,...
22273,2023-12-29,0.584449,0.587924,0.589654,0.605763,0.035673,BA,1.0
22274,2023-12-29,0.737537,0.729831,0.739415,0.798699,0.147066,INTC,1.0
22275,2023-12-29,0.979779,0.981421,0.987624,0.982597,0.058634,MSFT,1.0
22276,2023-12-29,0.990045,0.993200,1.000000,0.992263,0.012072,V,1.0


### Instantiate Environment

Using the `PortfolioOptimizationEnv`, it's easy to instantiate a portfolio optimization environment for reinforcement learning agents. In the example below, we use the dataframe created before to start an environment.

### Instantiate Model

Now, we can instantiate the model using FinRL API. In this example, we are going to use the EIIE architecture introduced by Jiang et. al.

:exclamation: **Note:** Remember to set the architecture's `time_window` parameter with the same value of the environment's `time_window`.

In [ ]:
df_portfolio = portfolio_norm_df[["date", "tic", "close", "high", "low"]]

from datetime import datetime

NUM_MODELS = 3


for split_date in TRAIN_DATES:
    df_portfolio_train = df_portfolio[(df_portfolio["date"] >= TRAIN_START_DATE) & (df_portfolio["date"] < split_date)]

    df_portfolio_test = df_portfolio[(df_portfolio["date"] > split_date) & (df_portfolio["date"] < TEST_END_DATE)]

    for i in range(0, NUM_MODELS):

        environment = PortfolioOptimizationEnv(
                df_portfolio_train,
                initial_amount=100000,
                comission_fee_pct=0.0025,
                time_window=50,
                features=["close", "high", "low"],
                normalize_df=None
            )

        # set PolicyGradient parameters
        model_kwargs = {
            "lr": 0.01,
            "policy": EIIE,
        }

        # here, we can set EIIE's parameters
        policy_kwargs = {
            "k_size": 3,
            "time_window": 50,
        }

        model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

        DRLAgent.train_model(model, episodes=40)

        current_timestamp = datetime.now()
        timestamp_string = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")

        torch.save(model.train_policy.state_dict(), f"policy_EIIE_{split_date}_{str(i)}_{timestamp_string}.pt")

        environment_test = PortfolioOptimizationEnv(
            df_portfolio_test,
            initial_amount=100000,
            comission_fee_pct=0.0025,
            time_window=50,
            features=["close", "high", "low"],
            normalize_df=None
        )

        EIIE_results = {
            "training": environment._asset_memory["final"],
            "test": {},
        }

        # instantiate an architecture with the same arguments used in training
        # and load with load_state_dict.
        policy = EIIE(time_window=50, device=device)
        policy.load_state_dict(torch.load("policy_EIIE.pt"))

        environment.reset()
        DRLAgent.DRL_validation(model, environment, policy=policy)
        EIIE_results["training"] = environment._asset_memory["final"]


        # 2020
        DRLAgent.DRL_validation(model, environment_test, policy=policy)
        EIIE_results["test"]["value"] = environment_test._asset_memory["final"]

        UBAH_results = {
            "train": {},
            "test": {},
        }

        PORTFOLIO_SIZE = len(TEST_TICKER)

        # train period
        terminated = False
        environment.reset()
        while not terminated:
            action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
            _, _, terminated, _ = environment.step(action)
        UBAH_results["train"]["value"] = environment._asset_memory["final"]

        # 2020
        terminated = False
        environment_test.reset()
        while not terminated:
            action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
            _, _, terminated, _ = environment_test.step(action)
        UBAH_results["test"]["value"] = environment_test._asset_memory["final"]


### Train Model

  0%|          | 0/40 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 1958396.5
Final accumulative portfolio value: 19.583965
Maximum DrawDown: -0.30520120295900544
Sharpe ratio: 1.1565652909453532


  2%|▎         | 1/40 [00:37<24:26, 37.59s/it]

Initial portfolio value:100000
Final portfolio value: 2673373.5
Final accumulative portfolio value: 26.733735
Maximum DrawDown: -0.3001341818144375
Sharpe ratio: 1.1769115049119097


  5%|▌         | 2/40 [01:23<27:03, 42.72s/it]

...

 95%|█████████▌| 38/40 [26:16<01:26, 43.18s/it]

Initial portfolio value:100000
Final portfolio value: 4087476.5
Final accumulative portfolio value: 40.874765
Maximum DrawDown: -0.3565325258093377
Sharpe ratio: 1.3730398519367388


 98%|█████████▊| 39/40 [26:54<00:41, 41.61s/it]

Initial portfolio value:100000
Final portfolio value: 3881257.5
Final accumulative portfolio value: 38.812575
Maximum DrawDown: -0.35653697422205255
Sharpe ratio: 1.3609118883892033


100%|██████████| 40/40 [27:29<00:00, 41.24s/it]


### Save Model

## Test Model

### Instantiate different environments

Since we have three different periods of time, we need three different environments instantiated to simulate them.

### Test EIIE architecture
Now, we can test the EIIE architecture in the three different test periods. It's important no note that, in this code, we load the saved policy even though it's not necessary just to show how to save and load your model.

### Test Uniform Buy and Hold
For comparison, we will also test the performance of a uniform buy and hold strategy. In this strategy, the portfolio has no remaining cash and the same percentage of money is allocated in each asset.

### Plot graphics

We can see that the agent is able to learn a good policy but its performance is worse the more the test period advances into the future. To get a better performance in 2022, for example, the agent should probably be trained again using more recent data.